In [1]:
BOOK_PATH = "ClassroomOfElite-Y1V1.epub"

In [2]:
import transformers
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup

/home/breno/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

book = epub.read_epub(BOOK_PATH)
chapters = list(book.get_items_of_type(ebooklib.ITEM_DOCUMENT))

def chapter_to_tokens(chapter: str, token_limit: int = 900):
    soup = BeautifulSoup(chapter.get_body_content(), 'html.parser')
    text = [para.get_text() for para in soup.find_all('p')]

    lent = 0
    token = ""
    tokens = []
    for para in text:
        lenp = len(para)
        if lent + lenp > token_limit:
            tokens.append(token)
            token = para
            lent = lenp
        else: 
            token += para
            lent += lenp


    return tokens

texts = {}
text = ""
for c in chapters[10:]:
    print(c.get_content())
    tokens = chapter_to_tokens(c)
    break
#     texts[c.get_name()] = chapter_to_str(c)]


print(tokens)
with open("original.txt", "w", encoding="utf-8") as f:
    f.write(" ".join(tokens))

/home/breno/.local/lib/python3.10/site-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')


b'<?xml version=\'1.0\' encoding=\'utf-8\'?>\n<!DOCTYPE html>\n<html xmlns="http://www.w3.org/1999/xhtml" xmlns:epub="http://www.idpf.org/2007/ops" epub:prefix="z3998: http://www.daisy.org/z3998/2012/vocab/structure/#" lang="en" xml:lang="en">\n  <head/>\n  <body><p class="P__STAR__STAR__STAR__page_break" id="auto_bookmark_toc_top"><span><span style=" font-weight: bold; font-size: 1.17em;">Chapter 4:<br/>These New First-Years are Nothing But Troublemakers</span></span></p>\n<p class="P_Prose_Formatting">\xc2\xa0</p>\n<p class="P_Prose_Formatting"><span style=" font-weight: bold; font-size: 2.40em;">T</span><span style=" font-size: 0.80em;">HERE WERE DOZENS</span><span style=" font-size: 1.00em;"> of first- and second-year students gathered in the gym. Most of the people there weren\xe2\x80\x99t second-years, but first-years. I supposed that many students saw this meet-and-greet as an important opportunity. </span></p>\n<p class="P_Prose_Formatting"><span style=" font-size: 1.00em;">Sin

In [4]:
len(text.split(" "))

1

In [5]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

def get_device():
    import torch 
    return "cuda:0" if torch.cuda.is_available() else "cpu"

device = get_device()

print("Loading in tokenizer")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")
print("Loading in model")
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")
print("Model is being loaded into:", device)
model.to(device);

Loading in tokenizer
Loading in model
Model is being loaded into: cuda:0


MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNor

In [8]:
def translate_token(text: str) -> str:
    model_inputs = tokenizer(text, return_tensors="pt").to(device)

    # translate from English to Hindi
    generated_tokens = model.generate(
        **model_inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id["pt_XX"]
    )

    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]


t_text = ""
curr, total = 1, len(tokens)
for token in tokens:
    print(f"{curr} / {total}")
    t_text += translate_token(token) + " "
    curr += 1
    # break


print(t_text)

with open("translated.txt", "w", encoding="utf-8") as f:
    f.write(t_text)


1 / 148
2 / 148
3 / 148
4 / 148
5 / 148
6 / 148
7 / 148
8 / 148
9 / 148
10 / 148
11 / 148
12 / 148
13 / 148
14 / 148
15 / 148
16 / 148
17 / 148
18 / 148
19 / 148
20 / 148
21 / 148
22 / 148
23 / 148
24 / 148
25 / 148
26 / 148
27 / 148
28 / 148
29 / 148
30 / 148
31 / 148
32 / 148
33 / 148
34 / 148
35 / 148
36 / 148
37 / 148
38 / 148
39 / 148
40 / 148
41 / 148
42 / 148
43 / 148
44 / 148
45 / 148
46 / 148
47 / 148
48 / 148
49 / 148
50 / 148
51 / 148
52 / 148
53 / 148
54 / 148
55 / 148
56 / 148
57 / 148
58 / 148
59 / 148
60 / 148
61 / 148
62 / 148
63 / 148
64 / 148
65 / 148
66 / 148
67 / 148
68 / 148
69 / 148
70 / 148
71 / 148
72 / 148
73 / 148
74 / 148
75 / 148
76 / 148
77 / 148
78 / 148
79 / 148
80 / 148
81 / 148
82 / 148
83 / 148
84 / 148
85 / 148
86 / 148
87 / 148
88 / 148
89 / 148
90 / 148
91 / 148
92 / 148
93 / 148
94 / 148
95 / 148
96 / 148
97 / 148
98 / 148
99 / 148
100 / 148
101 / 148
102 / 148
103 / 148
104 / 148
105 / 148
106 / 148
107 / 148
108 / 148
109 / 148
110 / 148
111 / 14

In [ ]:
with open("translated.txt", "r", encoding="utf-8") as f:
    t = f.read()
    t = t.replace("  ", "%&@")
    t = t.replace(" ", "")
    t = t.replace("%&@", " ")

with open("translated.txt", "w", encoding="utf-8") as f:
    f.write(t)